| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| S | array (Ny, Nx, 2, 2) | Rate of strain tensor over meshgrid |
| x_vortex | float | x-coordinate of vortex center |
| y_vortex | float | y-coordinate of vortex center |
| len_poincare | float | length of poincare map |
| len_threshold | float | maximum allowed perimeter of vortex |
| step_size | float | step size used for integration of shearline |
| sign_eta | float | sign determining orientation of closed shearline (+1/-1) |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/General_Functions")

In [2]:
# Import function which calculates eigenvalues/eigenvectors in 2D
from ipynb.fs.defs.loc_max import _loc_max
from ipynb.fs.defs.orient_vectorfield import _orient_vectorfield
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines
from ipynb.fs.defs.eigen import eigen_S

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

def _shearlines(X, Y, S, mu, x_vortex, y_vortex, len_poincare, len_threshold, step_size, sign_eta):
    
    s_1 = np.zeros((S.shape[0], S.shape[1]))*np.nan
    s_2 = np.zeros((S.shape[0], S.shape[1]))*np.nan
    v1 = np.zeros((S.shape[0], S.shape[1], 2))
    v2 = np.zeros((S.shape[0], S.shape[1], 2))
        
    # vector_field
    vector_field = np.zeros((S.shape[0], S.shape[1], 2))
    
    for i in range(s_1.shape[0]):
        for j in range(s_2.shape[1]):
            s_1[i, j], s_2[i, j], v1[i,j,:], v2[i,j,:] = eigen_S(S[i, j, :, :])
            if np.isfinite(s_1[i,j]):
                denominator = s_2[i, j] - s_1[i, j]
                if s_2[i,j]-mu >= 0 and mu-s_1[i,j]>=0:
                    vector_field[i,j,:] = np.sqrt((s_2[i,j]-mu)/denominator)*v1[i,j,:]+sign_eta*np.sqrt((mu-s_1[i,j])/denominator)*v2[i,j,:]
    
    # Defined domain
    defined_domain = np.isfinite(s_1).astype(int)
    
    # Set s_1/s_2 values to zero if they are 'nan'
    # This step is necessary for the gridded interpolation
    for i in range(s_1.shape[0]):
        for j in range(s_1.shape[1]):
            if np.isnan(s_1[i, j]):
                s_1[i, j] = 0
                s_2[i, j] = 0
    
    # Import Rectangular bivariate spline from scipy
    from scipy.interpolate import RectBivariateSpline as RBS
        
    # Interpolate Eigenvalue field
    interp_s = RBS(Y[:,0], X[0,:], s_1, kx = 1, ky = 1)
    
    # Define Poincare-section
    if x_vortex + len_poincare >= X[0,-1]:
        
        len_poincare = X[0,-1] - x_vortex
        
    x_pc = x_vortex + np.linspace(0, len_poincare, 100, endpoint = True)
    y_pc = y_vortex + np.linspace(0, 0, 100)
    
    # Define list of shearlines
    shearlines = [[], []]
    
    # Distance between intersection of poincare section with shearline and 
    # initial point from where shearline is computed on poincare section
    distance_ps_s = []
    
    # Iterate over points of poincare-section
    for i in tqdm(range(len(x_pc))):
        
        # Boolean intersection of shearline with poincare section
        bool_intersection = False
        
        shearline = [[], []]
        
        # Local maxima point
        x = np.array([x_pc[i], y_pc[i]])
        
        # Boolean integration
        bool_integration = True
        
        # Check orientation of vector-field and rieorient if needed.
        vx, vy = _orient_vectorfield(X, Y, x, vector_field) 
            
        # Initial vector orientation
        x_prime = np.array([vx, vy])
        
        if vy > 0:
            intersection_type = "anti-clock"
        else:
            intersection_type = "clock"
            
        # Initial length
        length = 0
        
        # Append point to shearline
        for ii in range(2):
            shearline[ii].append(x[ii])
            
        # Start Integration with dummy variable 's_array'
        while bool_integration:
                    
            # RK4 integration for tensorline
            x, x_prime = _RK4_tensorlines(X, Y, defined_domain, x, x_prime, step_size, vector_field, interp_s) 
            
            # If 'x' is defined then update length
            if x is not None:
                  
                # If x_prime is close to zero --> then we are close to a singularity of C
                # --> stop Integration
                
                if np.sqrt(x_prime[0]**2+x_prime[1]**2) < 10**(-6):
                    bool_integration = False
        
                # Compute length of shearline
                length += np.sqrt(x_prime[0]**2+x_prime[1]**2)*step_size
                
                # If length of shearline exceeds threshold
                # --> Stop integration
                if length > len_threshold:
                    bool_integration = False
        
                x1 = [shearline[0][-1], shearline[1][-1]]
                x2 = [x[0], x[1]]
                x3 = [x_pc[0], y_pc[0]]
                x4 = [x_pc[-1], y_pc[-1]]
                
                if intersection_type == "anti-clock":
                    Condition_intersection = x1[1] < x3[1] < x2[1]
                elif intersection_type == "clock":
                    Condition_intersection = x2[1] < x3[1] < x1[1]
                    
                if x3[0] <= x1[0] <= x4[0] and x3[0] < x2[0] < x4[0] and Condition_intersection:
                    
                    x_intersec = compute_intersection(x1, x2, x3, x4)
                
                    for j in range(2):
                        shearline[j].append(x_intersec[j])
                     
                    # check if closed shearline contains vortex center:
                    for j in range(len(shearline[0])-1):
                        
                        # compute vector tangent to closed shearline
                        tangent_vector = np.array([shearline[0][j+1]-shearline[0][j], shearline[1][j+1]-shearline[1][j]])
                        
                        # compute normal vector of closed shearline pointing outwards
                        if intersection_type == "anti-clock":
                            normal_vector = np.array([tangent_vector[1], -tangent_vector[0]])
                        elif intersection_type == "clock":
                            normal_vector = np.array([-tangent_vector[1], tangent_vector[0]])
                            
                        # compute vector pointing from vortex center to closed shearline
                        vector_center_to_shearline = np.array([shearline[0][j]-x_vortex, shearline[1][j]-y_vortex])
                    
                        # if for all points of the closed shearline
                        # the inner product between 'vector_center_to_shearline' and 
                        # 'normal_vector' is positive --y point is inside closed shearline'
                        # Otherwise, vortex center is not inside closed shearline
                        bool_vortex_in_closed_shearline = True
                        
                        if np.sign(vector_center_to_shearline[0]*normal_vector[0]+vector_center_to_shearline[1]*normal_vector[1]) <0:
                            bool_vortex_in_closed_shearline = False
                            break
                    
                    # If closed shearline satisfies all the above criteria, --> candidate for vortex boundary
                    if bool_vortex_in_closed_shearline:
                    
                        # compute distance between intersection of shearline with poincare-section 
                        # and initial point of shearline
                        distance_ps_s.append(shearline[0][-1]-shearline[0][0])
                    
                        for j in range(2):
                            shearlines[j].append(shearline[j])
                    
                        # Stop Integration!!!
                        bool_integration = False
                    
                # If there is no intersection --> Append point to shearline and continue integration 
                else:
                    for j in range(2):
                        shearline[j].append(x[j])
                    
            else:
                        
                bool_integration = False

        
    return shearlines, distance_ps_s, vector_field

In [3]:
def compute_intersection(x1, x2, x3, x4):
    px= ((x1[0]*x2[1]-x1[1]*x2[0])*(x3[0]-x4[0])-(x1[0]-x2[0])*(x3[0]*x4[1]-x3[1]*x4[0]))/((x1[0]-x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]-x4[0]) ) 
    py= ((x1[0]*x2[1]-x1[1]*x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]*x4[1]-x3[1]*x4[0]))/((x1[0]-x2[0])*(x3[1]-x4[1])-(x1[1]-x2[1])*(x3[0]-x4[0]) )
    return px, py